# FSL FEAT nipype

In [9]:
import os
import glob

import nipype
import nipype.interfaces.io as nio
import nipype.interfaces.fsl as fsl
import nipype.interfaces.ants as ants
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.algorithms.modelgen as model

## pipeline setup

In [10]:
# Which dataset are we looking at?
this_dataset = 'Leipzig_7T_SM'
# this_dataset = 'Leipzig_7T_GdH'
# this_dataset = 'NTNU_7T_SJSI'

In [11]:
# general set-up
base_dir = '/home/scotti/projects/3t_7t_sst_comparison'
work_dir = os.path.join(base_dir, 'processing', 'nipype_workflow_folders')
smooth_hp_dir = os.path.join(base_dir, 'derivatives', 'susan_smoothed_hp_func', this_dataset)

hpcutoff = 10000.   # in seconds
model_n = [0]
spaces = ['T1w']   # shouldn't touch this but just in case we _do_ want to go back to MNI....

if this_dataset == 'Leipzig_7T_SM':
    smoothing_fwhm = ['5p0']#['3p125']#['1p6']
    t_r = 3.0
#     numsubs = 18
    subject_ids = sorted([x.split('/')[-2].split('-')[-1] for x in glob.glob(os.path.join(smooth_hp_dir, 'sub-*/'))])
    numsubs = len(subject_ids)
    task = 'stop'
elif this_dataset == 'Leipzig_7T_GdH':
    smoothing_fwhm = ['1p5']
    t_r = 2.0
    numsubs = 20
    subject_ids = sorted([x.split('/')[-1].split('-')[-1] for x in glob.glob(os.path.join(smooth_hp_dir, 'sub-*T'))])
    task = 'stop'

template_brain = os.path.join(base_dir,'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')

In [12]:
print(f"""
this dataset : {this_dataset}
smoothing_fwhm : {smoothing_fwhm}
tr : {t_r}
number of subjects : {numsubs}
subject ids : {subject_ids}
smooth_hp_dir : {smooth_hp_dir}
model ns : {model_n}

""")


this dataset : Leipzig_7T_SM
smoothing_fwhm : ['5p0']
tr : 3.0
number of subjects : 15
subject ids : ['01', '02', '03', '05', '06', '07', '08', '09', '10', '11', '13', '15', '16', '17', '18']
smooth_hp_dir : /home/scotti/projects/3t_7t_sst_comparison/derivatives/susan_smoothed_hp_func/Leipzig_7T_SM
model ns : [0]




# first & second level models

In [13]:
def get_session_info(subject_id, run, task='stop', this_dataset='Leipzig_7T_SM', space='T1w', shift=-1.38/2, model_n=0,
                    root_dir='/home/scotti/projects/3t_7t_sst_comparison', include_physio=True):
    # other space: MNI152NLin2009cAsym
    import pandas as pd
    import numpy as np
    import os
    from nipype.interfaces.base import Bunch
            
    ### files ###
    sub = subject_id
    event_fn = f'{root_dir}/derivatives/event_files/{this_dataset}/sub-{sub}/func/sub-{sub}_task-{task}_run-{run}_events.tsv'
#         func_fn = f'{root_dir}/derivatives/fmriprep/fmriprep/sub-{sub}/ses-{ses}/func/sub-{sub}_ses-{ses}_task-{task}_run-{run}_space-{space}_desc-preproc_bold.nii.gz'
    confounds_fn = f'{root_dir}/derivatives/fmriprep/{this_dataset}/fmriprep/fmriprep/sub-{sub}/func/sub-{sub}_task-{task}_run-{run}_desc-confounds_timeseries.tsv'

    events = pd.read_csv(event_fn, sep='\t', index_col=None)
    events['duration'] = 0.001  # stick functions
    if model_n == 0:
        events = events.loc[events.trial_type.isin(['ss', 'fs', 'go'])]
    elif model_n == 1:
        events = events.loc[events.trial_type.isin(['response_left', 'response_right'])]
    events = events[['onset', 'trial_type', 'duration']]

    # slice time correction, nb: shift should be a negative number for STC
    events['onset'] += shift  

    ### confounds ###
    confounds = pd.read_csv(confounds_fn, sep='\t')
#        cosine_cols = [x for x in confounds.columns if 'cos' in x]
    include_confounds = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z', 'dvars', 'framewise_displacement'] #+ cosine_cols
    confounds = confounds[include_confounds].fillna(method='bfill')

    # get retroicor
    if include_physio:
#         retroicor_fn = f'{root_dir}/derivatives/retroicor/sub-{sub}/ses-{ses}/func/sub-{sub}_ses-{ses}_task-{task}_run-{run}_desc-retroicor_regressors.tsv'
#         if not os.path.exists(retroicor_fn):
        ## take first 20 aCompCor components
        print("No retroicor found, including 20 a_comp_cor components")
        a_comp_cor = pd.read_csv(confounds_fn, sep='\t')[['a_comp_cor_' + str(x).zfill(2) for x in range(20)]]
        confounds = pd.concat([confounds, a_comp_cor], axis=1)
#         else:
#             retroicor = pd.read_csv(retroicor_fn, sep='\t', header=None).iloc[:,:20]  ## 20 components in total
#             retroicor.columns = ['cardiac_' + str(x) for x in range(6)] + ['respiratory_' + str(x) for x in range(8)] + ['respiratoryxcardiac_' + str(x) for x in range(4)] + ['HRV', 'RVT']
#             confounds = pd.concat([confounds, retroicor], axis=1)

    # save the confounds that we actually include in the GLM to csv, no header or index
    confounds_fn = f'{root_dir}/derivatives/confounds/{this_dataset}/sub-{sub}/func/sub-{sub}_task-{task}_run-{run}_desc-confounds_timeseries.txt'
    os.makedirs(os.path.dirname(confounds_fn), exist_ok=True)
    confounds.to_csv(confounds_fn, sep='\t', header=False, index=False)

    ### Make bunch ###
    if model_n == 0:
        conditions=['fs',
                    'ss',
                    'go']

    elif model_n ==1:
        conditions = [
                     'response_left',
                     'response_right'
                     ]

    onsets = [events.loc[events.trial_type==trial_type,'onset'].tolist() for trial_type in conditions]
    durations = [events.loc[events.trial_type==trial_type,'duration'].tolist() for trial_type in conditions]
    amplitudes = [[1] * events.loc[events.trial_type==trial_type,'onset'].shape[0] for trial_type in conditions]
#             amplitudes = [events.loc[events.trial_type==trial_type,'modulation'].tolist() for trial_type in conditions]

    info = Bunch(conditions=conditions,
                 onsets=onsets,
                 durations=durations,
                 amplitudes=amplitudes)
#         print(info)
    
    # always return info and the confounds_fn
    return info, confounds_fn #, contrasts[0]

if model_n == [0]:
    contrasts = [('fs', 'T', ['fs'], [1.0]),
                 ('ss', 'T', ['ss'], [1.0]),
                 ('go', 'T', ['go'], [1.0]),
                 ('fs-go', 'T', ['fs', 'go'], [1, -1]),
                 ('fs-ss', 'T', ['fs', 'ss'], [1, -1]),
                 ('ss-go', 'T', ['ss', 'go'], [1, -1]),
                 ]
    
elif model_n == [1]:
    contrasts = [('response_left', 'T', ['response_left'], [1.0]),
                 ('response_right', 'T', ['response_right'], [1.0]),
                 ('left-right', 'T', ['response_left','response_right'], [1,-1])
                 ]

In [14]:
def get_runs_per_sub(subject_id):
    # here we can add the runs per subject if some subject miss a run or two
    if not subject_id in ['17']:
        runs = [1,2,3]
    else:
        runs = [1,2]
    
    return runs

In [15]:
workflow = pe.Workflow(name='feat_level12_sst_hp')
workflow.base_dir = os.path.join(base_dir, 'processing', 'nipype_workflow_folders', this_dataset)
workflow.config = {"execution": {"crashdump_dir":os.path.join(base_dir, 'processing', 'crashdumps')}}

# identity
identity = pe.Node(util.IdentityInterface(fields=['subject_id', 'space', 'model_n', 'smoothing_fwhm']), name='identity')
identity.iterables = [('subject_id', subject_ids),
                      ('space', spaces),
                      ('model_n', model_n),
                      ('smoothing_fwhm', smoothing_fwhm)]

# selector
# if running only one run, copy the mask from run 1 to run 2, 'mask' variable below needs a list, it will crash if only one is found
#(terrible way to get around this.. fix at some point.. lazy..)
templates = {'smoothed_funcs': os.path.join(base_dir, 'derivatives', 'susan_smoothed_hp_func', this_dataset,
                                            'sub-{subject_id}', 'func',  
                                            f'sub-{{subject_id}}_task-{task}_run-*_space-{{space}}*_fwhm-{{smoothing_fwhm}}_tempfilt.nii.gz'),
             'mask': os.path.join(base_dir, 'derivatives', 'fmriprep', this_dataset, 'fmriprep', 'fmriprep',
                                  'sub-{subject_id}', 'func', 
                                  f'sub-{{subject_id}}_task-{task}_run-*_space-{{space}}_desc-brain_mask.nii.gz'),
             'composite':  os.path.join(base_dir, 'derivatives', 'fmriprep', this_dataset, 'fmriprep', 'fmriprep',
                                        'sub-{subject_id}', 'anat', 
                                        'sub-{subject_id}_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5'),
             'xfm': os.path.join(base_dir, 'derivatives', 'fmriprep', this_dataset, 'fmriprep', 'fmriprep',
                                  'sub-{subject_id}', 'func', 
                                  f'sub-{{subject_id}}_task-{task}_run-*_from-scanner_to-T1w_mode-image_xfm.txt'),}
selector = pe.Node(nio.SelectFiles(templates), name='selector')

workflow.connect(identity, 'subject_id', selector, 'subject_id')
workflow.connect(identity, 'space', selector, 'space')
workflow.connect(identity, 'smoothing_fwhm', selector, 'smoothing_fwhm')

# get run info per sub
run_info_getter = pe.Node(util.Function(function=get_runs_per_sub,
                                        input_names=['subject_id'],
                                        output_names=['runs']), name='run_info_getter')
workflow.connect(identity, 'subject_id', run_info_getter, 'subject_id')

# session info getter
session_info_getter = pe.MapNode(util.Function(function=get_session_info,
                                     input_names=['subject_id', 'run', 'task', 'this_dataset', 'space', 'shift', 'model_n'],
                                     output_names=['session_info', 'confounds']),
                                 iterfield=['run'],
                                 name='session_info_getter')
session_info_getter.inputs.task=task
session_info_getter.inputs.this_dataset=this_dataset
session_info_getter.inputs.space='T1w'
session_info_getter.inputs.shift = -t_r/2

workflow.connect(identity, 'subject_id', session_info_getter, 'subject_id')
workflow.connect(run_info_getter, 'runs', session_info_getter, 'run')
workflow.connect(identity, 'model_n', session_info_getter, 'model_n')


# model setup
specifymodel = pe.Node(model.SpecifyModel(), name='specifymodel1')
specifymodel.inputs.input_units = 'secs'
specifymodel.inputs.time_repetition = t_r
specifymodel.inputs.high_pass_filter_cutoff = hpcutoff

workflow.connect(session_info_getter, 'session_info', specifymodel, 'subject_info')
## old flow (with cosines): immediately connect to specifymodel
workflow.connect(selector, 'smoothed_funcs', specifymodel, 'functional_runs')

# Level 1 design
level1design = pe.Node(interface=fsl.Level1Design(), name="level1design")
level1design.inputs.interscan_interval = t_r
level1design.inputs.bases = {'dgamma': {'derivs': True}}
level1design.inputs.contrasts = contrasts
level1design.inputs.model_serial_correlations = True

workflow.connect(specifymodel, 'session_info', level1design, 'session_info')
# workflow.connect(session_info_getter, 'contrasts', level1design, 'contrasts')

# FEAT model
modelgen = pe.MapNode(interface=fsl.FEATModel(), iterfield=['ev_files', 'fsf_file', 'args'], name='modelgen')

workflow.connect(level1design, 'ev_files', modelgen, 'ev_files')
workflow.connect(level1design, 'fsf_files', modelgen, 'fsf_file')
workflow.connect(session_info_getter, 'confounds', modelgen, 'args')   # add confounds here


# FILM GLS
iterfield = ['design_file', 'in_file', 'tcon_file']
modelestimate = pe.MapNode(interface=fsl.FILMGLS(smooth_autocorr=True,
                                                 mask_size = 5,
                                                 threshold = 1000),  # 
                          name='modelestimate',
                          iterfield=iterfield,
                          mem_gb=10)

# ## old flow (with cosines): immediately connect to modelestimate
workflow.connect(selector, 'smoothed_funcs', modelestimate, 'in_file')
# # new flow: connect high-passed data
# workflow.connect(highpass, 'out_file', modelestimate, 'in_file')


workflow.connect(modelgen, 'design_file', modelestimate, 'design_file')
workflow.connect(modelgen, 'con_file', modelestimate, 'tcon_file')


#### Fixed effects
# merge copes, varcopes
copemerge = pe.MapNode(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="copemerge")

varcopemerge = pe.MapNode(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="varcopemerge")

maskemerge = pe.MapNode(interface=fsl.Merge(dimension='t'),
                       iterfield=['in_files'],
                       name="maskemerge")

def sort_copes(files):
    numelements = len(files[0])
    outfiles = []
    for i in range(numelements):
        outfiles.insert(i,[])
        for j, elements in enumerate(files):
            outfiles[i].append(elements[i])
    return outfiles

workflow.connect(modelestimate, ('copes',sort_copes), copemerge, 'in_files')
workflow.connect(modelestimate, ('varcopes',sort_copes), varcopemerge, 'in_files')


level2model = pe.Node(interface=fsl.L2Model(), name='l2model')
def num_copes(files):
    return len(files)
workflow.connect(modelestimate, ('copes',num_copes), level2model, 'num_copes')


pickfirst = lambda x: x[0]
flameo = pe.MapNode(
    interface=fsl.FLAMEO(run_mode='fe'),
    name="flameo",
    iterfield=['cope_file', 'var_cope_file'])

workflow.connect([
    (selector, flameo, [(('mask', pickfirst), 'mask_file')]),
    (copemerge, flameo, [('merged_file', 'cope_file')]),
    (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
    (level2model, flameo, [('design_mat', 'design_file'),
                           ('design_con', 't_con_file'), 
                           ('design_grp', 'cov_split_file')]),
])

## Warp to MNI-space
def warp_files(copes, varcopes, zstats, tdofs, mat, template_brain):
    # stolen from https://dpaniukov.github.io/2016/07/14/three-level-analysis-with-fsl-and-ants-2.html
    import nipype.interfaces.ants as ants

    out_copes = []
    out_varcopes = []
    out_zstats = []
    out_tdofs = []
    
    warp = ants.ApplyTransforms()
    warp.inputs.input_image_type = 0
    warp.inputs.interpolation = 'Linear'
    warp.inputs.invert_transform_flags = [False] #,False]
    warp.inputs.reference_image = template_brain
    warp.inputs.transforms = mat

    if not isinstance(copes, list):
        copes = [copes]
        varcopes = [varcopes]
        zstats = [zstats]
        tdofs = [tdofs]
    
    for cope in copes:
        warp.inputs.input_image = cope
        res=warp.run()
        out_copes.append(str(res.outputs.output_image))

    for varcope in varcopes:
        warp.inputs.input_image = varcope
        res=warp.run()
        out_varcopes.append(str(res.outputs.output_image))
        
    for zstat in zstats:
        warp.inputs.input_image = zstat
        res=warp.run()
        out_zstats.append(str(res.outputs.output_image))
    
    for tdof in tdofs:
        warp.inputs.input_image = tdof
        res=warp.run()
        out_tdofs.append(str(res.outputs.output_image))

    return out_copes, out_varcopes, out_zstats, out_tdofs

warpfunc = pe.MapNode(util.Function(input_names=['copes', 'varcopes', 'zstats', 'tdofs', 'mat','template_brain'],
                               output_names=['out_copes', 'out_varcopes', 'out_zstats', 'out_tdofs'],
                               function=warp_files),
                               iterfield=['copes', 'varcopes', 'zstats', 'tdofs'],
                  name='warpfunc')

warpfunc.inputs.template_brain = template_brain
workflow.connect(flameo, 'copes', warpfunc, 'copes')
workflow.connect(flameo, 'var_copes', warpfunc, 'varcopes')
workflow.connect(flameo, 'zstats', warpfunc, 'zstats')
workflow.connect(flameo, 'tdof', warpfunc, 'tdofs')
workflow.connect(selector, 'composite', warpfunc, 'mat')

#### post-FE analyis
# Smoothness estimation
smoothestimate = pe.MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], name='smoothestimate')

workflow.connect(selector, ('mask', pickfirst), smoothestimate, 'mask_file')
workflow.connect(flameo, 'zstats', smoothestimate, 'zstat_file')

#
get_volume = pe.Node(fsl.ImageStats(op_string = '-V'), name='get_volume')
workflow.connect(selector, ('mask', pickfirst), get_volume, 'in_file')

# Cluster threshold
grf_cluster = pe.MapNode(fsl.Cluster(), iterfield=['dlh', 'in_file'], name='grf_cluster')
grf_cluster.iterables = [("threshold", [2.3, 3.1])] 
workflow.connect(smoothestimate, 'dlh', grf_cluster, 'dlh')
workflow.connect(flameo, 'zstats', grf_cluster, 'in_file')

def convert_volume(input):
    return int(input[1])

workflow.connect(get_volume, ('out_stat', convert_volume), grf_cluster, 'volume')
grf_cluster.inputs.out_threshold_file = True


## datasink
ds = pe.Node(nio.DataSink(), name='datasink')
ds.inputs.base_directory = os.path.join(base_dir, 'derivatives', 'glm_feat_hp_sct')

#_model_n_0_smoothing_fwhm_1p5_space_T1w_subject_id_002
substitutions = [(f't1w/level2_{stat_type}s/_model_n_{model_n_}_smoothing_fwhm_{fwhm}_space_T1w_subject_id_{sub}/_flameo{contrast_n}/{stat_type}1.nii.gz',
                  f'{this_dataset}/sub-{sub}/func/fwhm-{fwhm}/model-{model_n_}/sub-{sub}_task-{task}_space-T1w_model-{model_n_}_contrast-{contrast_n}_desc-{stat_type}.nii.gz')
                  for sub in subject_ids
                  for contrast_n in range(len(contrasts))
                  for model_n_ in model_n
                  for fwhm in smoothing_fwhm
                  for stat_type in ['cope', 'zstat', 'varcope', 'tdof_t']
                  ]


substitutions += [(f'mni/level2_{stat_type}s/_model_n_{model_n_}_smoothing_fwhm_{fwhm}_space_T1w_subject_id_{sub}/_warpfunc{contrast_n}/{stat_type}1_trans.nii.gz',
                   f'{this_dataset}/sub-{sub}/func/fwhm-{fwhm}/model-{model_n_}/sub-{sub}_task-{task}_space-MNI152NLin2009cAsym_model-{model_n_}_contrast-{contrast_n}_desc-{stat_type}.nii.gz')
                  for sub in subject_ids
                  for contrast_n in range(len(contrasts))
                  for model_n_ in model_n
                  for fwhm in smoothing_fwhm
                  for stat_type in ['cope', 'zstat', 'varcope', 'tdof_t']
                  ]

ds.inputs.substitutions = substitutions


## cluster thresholds: leave for now, this is more of a third-level model thing anyway
# workflow.connect(grf_cluster, 'threshold_file', ds, 'grf_thresholded_zstats_file')
# workflow.connect(grf_cluster, 'locnalmax_txt_file', ds, 'grf_localmax_txt_file')

## T1w-space
workflow.connect(flameo, 'zstats', ds, 'subject_level_model.t1w.level2_zstats')
workflow.connect(flameo, 'copes', ds, 'subject_level_model.t1w.level2_copes')
workflow.connect(flameo, 'var_copes', ds, 'subject_level_model.t1w.level2_varcopes')
workflow.connect(flameo, 'tdof', ds, 'subject_level_model.t1w.level2_tdof_ts')

## MNI-space
workflow.connect(warpfunc, 'out_zstats', ds, 'subject_level_model.mni.level2_zstats')
workflow.connect(warpfunc, 'out_copes', ds, 'subject_level_model.mni.level2_copes')
workflow.connect(warpfunc, 'out_varcopes', ds, 'subject_level_model.mni.level2_varcopes')
workflow.connect(warpfunc, 'out_tdofs', ds, 'subject_level_model.mni.level2_tdof_ts')
# workflow.connect(flameo, 'tdof', ds, 'level2_tdof')

In [ ]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs':10, 'memory_gb': 100})

220923-15:10:37,603 nipype.workflow INFO:
	 Workflow feat_level12_sst_hp settings: ['check', 'execution', 'logging', 'monitoring']
220923-15:10:38,122 nipype.workflow INFO:
	 Running in parallel.
220923-15:10:38,127 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 30 jobs ready. Free memory (GB): 100.00/100.00, Free processors: 10/10.
220923-15:10:38,220 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_hp.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_5p0_space_T1w_subject_id_01/selector".
220923-15:10:38,220 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_hp.run_info_getter" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_5p0_space_T1w_subject_id_01/run_info_getter".
220923-15:10:38,220 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_hp.selector" 

220923-15:10:40,216 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_hp.run_info_getter" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_5p0_space_T1w_subject_id_09/run_info_getter".
220923-15:10:40,217 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_hp.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_5p0_space_T1w_subject_id_10/selector".
220923-15:10:40,217 nipype.workflow INFO:
	 [Node] Executing "selector" <nipype.interfaces.io.SelectFiles>
220923-15:10:40,217 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_hp.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_5p0_space_T1w_subject_id_11/selector".
220923-15:10:40,219 nipype.workflow INFO:
	 [Node] Execut

220923-15:10:42,220 nipype.workflow INFO:
	 [Node] Finished "run_info_getter", elapsed time 0.000394s.
220923-15:10:42,222 nipype.workflow INFO:
	 [Node] Executing "run_info_getter" <nipype.interfaces.utility.wrappers.Function>
220923-15:10:42,224 nipype.workflow INFO:
	 [Node] Finished "run_info_getter", elapsed time 0.000403s.
220923-15:10:42,234 nipype.workflow INFO:
	 [Node] Finished "selector", elapsed time 0.01648s.
220923-15:10:42,238 nipype.workflow INFO:
	 [Node] Finished "selector", elapsed time 0.024771s.
220923-15:10:42,241 nipype.workflow INFO:
	 [Node] Finished "selector", elapsed time 0.027061s.
220923-15:10:42,246 nipype.workflow INFO:
	 [Node] Finished "selector", elapsed time 0.031566s.
220923-15:10:42,247 nipype.workflow INFO:
	 [Node] Finished "selector", elapsed time 0.02705s.
220923-15:10:44,132 nipype.workflow INFO:
	 [Job 20] Completed (feat_level12_sst_hp.selector).
220923-15:10:44,133 nipype.workflow INFO:
	 [Job 21] Completed (feat_level12_sst_hp.run_info_get

220923-15:10:46,211 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_hp.get_volume" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_5p0_space_T1w_subject_id_16/get_volume".
220923-15:10:46,211 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_hp.get_volume" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_5p0_space_T1w_subject_id_17/get_volume".
220923-15:10:46,212 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_hp.get_volume" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_5p0_space_T1w_subject_id_18/get_volume".
220923-15:10:46,212 nipype.workflow INFO:
	 [Node] Setting-up "feat_level12_sst_hp.get_volume" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workfl

220923-15:10:48,217 nipype.workflow INFO:
	 [Node] Setting-up "_session_info_getter0" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_5p0_space_T1w_subject_id_05/session_info_getter/mapflow/_session_info_getter0".
220923-15:10:48,217 nipype.workflow INFO:
	 [Node] Executing "_session_info_getter2" <nipype.interfaces.utility.wrappers.Function>
220923-15:10:48,218 nipype.workflow INFO:
	 [Node] Setting-up "_session_info_getter1" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_5p0_space_T1w_subject_id_05/session_info_getter/mapflow/_session_info_getter1".
220923-15:10:48,218 nipype.workflow INFO:
	 [Node] Cached "_session_info_getter0" - collecting precomputed outputs
220923-15:10:48,219 nipype.workflow INFO:
	 [Node] Setting-up "_session_info_getter2" in "/home/scotti/projects/3t_7t_sst_comparison/proce

...

220923-15:11:14,272 nipype.workflow INFO:
	 [Node] Setting-up "_modelgen0" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_5p0_space_T1w_subject_id_16/modelgen/mapflow/_modelgen0".
220923-15:11:14,273 nipype.workflow INFO:
	 [Node] Executing "_modelestimate1" <nipype.interfaces.fsl.model.FILMGLS>
220923-15:11:14,274 nipype.workflow INFO:
	 [Node] Setting-up "_modelgen1" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_5p0_space_T1w_subject_id_16/modelgen/mapflow/_modelgen1".
220923-15:11:14,275 nipype.workflow INFO:
	 [Node] Executing "_modelestimate2" <nipype.interfaces.fsl.model.FILMGLS>
220923-15:11:14,276 nipype.workflow INFO:
	 [Node] Setting-up "_modelgen2" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level12_sst_hp/_model_n_0_smoothing_fwhm

220923-15:11:18,248 nipype.workflow INFO:
	 [Node] Setting-up "_modelgen0" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_5p0_space_T1w_subject_id_18/modelgen/mapflow/_modelgen0".
220923-15:11:18,255 nipype.workflow INFO:
	 [Node] Cached "_modelgen0" - collecting precomputed outputs
220923-15:11:18,255 nipype.workflow INFO:
	 [Node] "_modelgen0" found cached.
220923-15:11:18,256 nipype.workflow INFO:
	 [Node] Cached "_modelgen0" - collecting precomputed outputs
220923-15:11:18,257 nipype.workflow INFO:
	 [Node] "_modelgen0" found cached.
220923-15:11:18,257 nipype.workflow INFO:
	 [Node] Setting-up "_modelgen1" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_5p0_space_T1w_subject_id_17/modelgen/mapflow/_modelgen1".
220923-15:11:18,258 nipype.workflow INFO:
	 [Node] Setting-up "_modelgen1" in "/home/s

220923-15:41:47,907 nipype.workflow INFO:
	 [MultiProc] Running 10 tasks, and 32 jobs ready. Free memory (GB): 0.00/100.00, Free processors: 0/10.
                     Currently running:
                       * _modelestimate2
                       * _modelestimate1
                       * _modelestimate2
                       * _modelestimate1
                       * _modelestimate0
                       * _modelestimate1
                       * _modelestimate0
                       * _modelestimate2
                       * _modelestimate1
                       * _modelestimate0
220923-15:42:15,956 nipype.workflow INFO:
	 [Node] Finished "_modelestimate0", elapsed time 1859.683106s.
220923-15:42:17,934 nipype.workflow INFO:
	 [Job 346] Completed (_modelestimate0).
220923-15:42:17,936 nipype.workflow INFO:
	 [MultiProc] Running 9 tasks, and 32 jobs ready. Free memory (GB): 10.00/100.00, Free processors: 1/10.
                     Currently running:
                       * _m

220923-15:47:02,289 nipype.workflow INFO:
	 [Node] Executing "_modelestimate2" <nipype.interfaces.fsl.model.FILMGLS>
220923-15:47:04,209 nipype.workflow INFO:
	 [MultiProc] Running 10 tasks, and 41 jobs ready. Free memory (GB): 0.00/100.00, Free processors: 0/10.
                     Currently running:
                       * _modelestimate2
                       * _modelestimate1
                       * _modelestimate0
                       * _modelestimate2
                       * _modelestimate1
                       * _modelestimate1
                       * _modelestimate0
                       * _modelestimate2
                       * _modelestimate1
                       * _modelestimate0
220923-15:47:33,58 nipype.workflow INFO:
	 [Node] Finished "_modelestimate0", elapsed time 2180.756619s.
220923-15:47:34,238 nipype.workflow INFO:
	 [Job 335] Completed (_modelestimate0).
220923-15:47:34,242 nipype.workflow INFO:
	 [MultiProc] Running 9 tasks, and 41 jobs ready. Free m

220923-15:47:58,354 nipype.workflow INFO:
	 [Node] Setting-up "_modelestimate0" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level12_sst_hp/_model_n_0_smoothing_fwhm_5p0_space_T1w_subject_id_08/modelestimate/mapflow/_modelestimate0".
220923-15:47:58,361 nipype.workflow INFO:
	 [Node] Executing "_modelestimate0" <nipype.interfaces.fsl.model.FILMGLS>
220923-15:48:00,265 nipype.workflow INFO:
	 [MultiProc] Running 10 tasks, and 49 jobs ready. Free memory (GB): 0.00/100.00, Free processors: 0/10.
                     Currently running:
                       * _modelestimate0
                       * _modelestimate2
                       * _modelestimate1
                       * _modelestimate0
                       * _modelestimate2
                       * _modelestimate1
                       * _modelestimate0
                       * _modelestimate2
                       * _modelestimate1
                       * _modelestima

In [8]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/scotti/projects/3t_7t_sst_comparison/processing/crashdumps/crash-20220728-174554-scotti-_modelgen2-7ffee7d4-2bae-4519-9ecd-bf5add208a01.pklz')
res

ERROR! Session/line number was not unique in database. History logging moved to new session 275


{'node': _modelgen2,
 'traceback': ['Traceback (most recent call last):\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node\n    result["result"] = node.run(updatehash=updatehash)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 521, in run\n    result = self._run_interface(execute=True)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 639, in _run_interface\n    return self._run_command(execute)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 750, in _run_command\n    raise NodeExecutionError(\n',
  'nipype.pipeline.engine.nodes.NodeExecutionError: Exception raised while executing Node _modelgen2.\n\nTraceback (most recent call last):\n  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/interfaces/base/core.py", line 401, in run\n    outputs = sel

In [8]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/scotti/projects/3t_7t_sst_comparison/processing/crashdumps/crash-20220725-201456-scotti-_modelgen2-853c3a8d-a5d2-434c-9c99-ac3b0735bd4a.pklz')
res

{'node': _modelgen2,
 'traceback': ['Traceback (most recent call last):\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node\n    result["result"] = node.run(updatehash=updatehash)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 521, in run\n    result = self._run_interface(execute=True)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 639, in _run_interface\n    return self._run_command(execute)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 750, in _run_command\n    raise NodeExecutionError(\n',
  'nipype.pipeline.engine.nodes.NodeExecutionError: Exception raised while executing Node _modelgen2.\n\nTraceback (most recent call last):\n  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/interfaces/base/core.py", line 401, in run\n    outputs = sel

ERROR! Session/line number was not unique in database. History logging moved to new session 260


In [9]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/scotti/projects/3t_7t_sst_comparison/processing/crashdumps/crash-20220725-201450-scotti-_modelgen0-f309bcf2-f7a0-42a2-8b46-32fb7dce39b4.pklz')
res

{'node': _modelgen0,
 'traceback': ['Traceback (most recent call last):\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node\n    result["result"] = node.run(updatehash=updatehash)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 521, in run\n    result = self._run_interface(execute=True)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 639, in _run_interface\n    return self._run_command(execute)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 750, in _run_command\n    raise NodeExecutionError(\n',
  'nipype.pipeline.engine.nodes.NodeExecutionError: Exception raised while executing Node _modelgen0.\n\nTraceback (most recent call last):\n  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/interfaces/base/core.py", line 401, in run\n    outputs = sel